# Exercício 3 - Análise de sentimento no Twitter

Nesse exercício vamos analisar uma base de dados do Twitter. Vamos também aplicar um classificador deep learning em nossa base e comparar com um classificador mais básico.

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import SimpleRNN, LSTM, GRU

Usaremos a base de dados Sentiment140, com sentimentos de tweets organizada pela Universidade de Stanford. Informações sobre a base podem ser encontradas no link (http://help.sentiment140.com/for-students). O formato da base no link é diferente do formato que usaremos nesse exercício, mas os dados são os mesmos. Assim, utilize a base de dados do Blackboard ou do Google Drive. É recomendado que o exercício seja executado no Google Colab, utilizando GPU ou TPU para o kernel, já que usaremos o Keras. 

É necessário descompactar o arquivo. Caso esteja executando no Google Colab, execute o procedimento de montagem do drive no Colab e descompacte o arquivo.

Vamos carregar a base de treinamento. Lembre-se de informar o caminho do arquivo no comando abaixo. Aqui consideramos que o arquivo de treinamento está disponível na pasta data.

In [ ]:
cols = ['sentiment','id','date','query_string','user','text']
training_df = pd.read_csv("./data/training.1600000.processed.noemoticon.csv",header=None, names=cols, encoding = "ISO-8859-1")

Vamos definir alguns parâmetros importantes. Primeiro definimos a quantidade de itens que vamos utilizar da base. A base possui 160000 registros, o que é pesado para analisarmos com deep learning. Definimos também a quantidade máxima de palavras que consideraremos na tokenização. O processo de tokenização que usaremos cria um dicionário de palavras e retorna um vetor com o índice de calada palavra no dicionário. Como estamos lidando com redes sociais, temos diversas palavras únicas, o que criaria um dicionário muito grande, por isso limitamos esse dicionário. Por fim, definimos o tamanho máximo de cada sentença, a fim de limitar o tamanho de cada vetor.

In [ ]:
data_limit = 200000
max_words = 100000
max_len = 200

Vamos tratar nossos dados, removendo html e caracteres estranhos. Para isso criamos uma função que trata cada tweet em separado. Depois aplicamos essa função em cada tweet, já limitando ao nosso limite definido acima. Vamos aproveitar e embaralhar os dados, já que as classes estão ordenadas no dataset. 

In [ ]:
training_df.drop(['id','date','query_string','user'],axis=1,inplace=True)
train_df = training_df.sample(frac=1).reset_index(drop=True)

In [ ]:
import re
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    return lower_case

In [ ]:
clean_training_texts = []
for i in range(0, data_limit):
    if( (i+1)%100000 == 0 ):
        print("Tweets %d of %d has been processed" % ( i+1, data_limit ))
    clean_training_texts.append(tweet_cleaner(train_df['text'][i]))

Vamos tokenizar os tweets utilizando o Tokenizador do Keras. Esse tokenizador tem a vantagem de retornar sequências de números ao invés da sentença tokenizada. Essa sequência possuí índices das palavras em um dicionário com todas as palavras na lista. Isso é importante para que nossa rede possa treinar embeddings para as palavras. Aqui limitamos a quantidade de palavras no dicionário pelo parâmetro max_words

In [ ]:
tokenizer = Tokenizer(num_words=max_words,lower=True, split=" ")
tokenizer.fit_on_texts(clean_training_texts)
tokens = tokenizer.texts_to_sequences(clean_training_texts)

Vamos limitar nossas sequências para um número definido de valores, adicionando zeros caso a sequência seja menor ou cortando a sequência caso seja maior

In [ ]:
X = sequence.pad_sequences(tokens, maxlen=max_len)

Vamor gerar nosso conjunto de labels (y). Para isso, retiramos do dataframe inicial os registros da coluna sentiment limitando ao parâmetro data_limit. Fazemos um tratamento no conjunto para que esse seja binário. No dataset original, o rótulo 0 indica negativo e o rótulo 4 indica positivo.

In [ ]:
y = train_df.sentiment[:data_limit]
y = [1 if v == 4 else 0 for v in y]

Agora vamos criar nossa rede de classificação. Para isso, crie uma rede recorrente que classifique nossos tweets. A rede deve obrigatoriamente ter uma camada de embedding, uma camada recorrente (SimpleRNN, GRU ou LSTM) e a camada de saída da classificação (camada densa com 1 unidade, de preferência com ativação sigmoide). Você está livre para criar quantas camadas desejar, lembrando que quanto mais camadas, mais demorado é o treinamento da rede.

In [ ]:
model = Sequential()

#Crie sua rede aqui


In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

Vamos treinar nossa rede. Aqui está definido alguns parâmetros para treinamento, sinta-se a vontade para alterar esses parâmetros conforme desejar

In [ ]:
history = model.fit(X, y, epochs=10, batch_size=128, validation_split=0.25)

Verifique a perda e a acurácia de validação durante o treinamento. Verifique se seu modelo está performando bem com os parâmetros e arquitetura de rede escolhida. Caso deseje, utilize o history para mostrar gráficos com os dados do treinamento.

In [ ]:
#Espaço para gráficos caso deseje

Vamos comparar com um classificador clássico, a Regressão Logística. Primeiramente vamos vetorizar o texto usando o TF-IDF. 

In [ ]:
#complete as linhas com o TfidfVectorizer
tfidf = 
X = 

Separe em conjunto de teste e validação, usando o train_test_split. Aqui vamos manter a mesma taxa de validação (test_size) usada ao treinar a rede recorrente.

In [ ]:
#complete a linha abaixo
X_train, X_val, y_train, y_val = 

Crie a regressão logistica e treine o modelo.

In [ ]:
#complete as linhas abaixo
lr = 

Vamos agora ver a acurácia do modelo. A acurácia é muito diferente do modelo deep learning?

In [ ]:
print ("Accuracy : %s" % (accuracy_score(y_val, lr.predict(X_val))))